# Import Libraries

In [1]:
import glob
import pickle
from lesson_functions import *
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from scipy.ndimage.measurements import label
import collections as col
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
%matplotlib inline

from functions import *

# Load the SVM

In [2]:
# Load the classifier and parameters
data_file = 'svm.p'
with open(data_file, mode='rb') as f:
    data = pickle.load(f)
    
svc = data['svc'] 
X_scaler = data['X_scaler']
color_space = data['color_space']
spatial_size = data['spatial_size']
hist_bins = data['hist_bins']
orient = data['orient']
pix_per_cell = data['pix_per_cell']
cell_per_block = data ['cell_per_block']
hog_channel = data['hog_channel']
spatial_feat = data ['spatial_feat']
hist_feat = data['hist_feat']
hog_feat = data['hog_feat']

# Define Vehicle Detection Functions

In [3]:
def search(image):
    # Grab the shape of the incoming image
    shape = np.shape(image)
    
    # Define the relative size of each of the boxes to look for vehicles
    window0 = int(.333*shape[0])
    window1 = int(.25*shape[0])
    window2 = int(.167*shape[0])
    window3 = int(.097*shape[0])
    
    # Define how much overlap there is for the windows
    o_const = .75
    window = [(window0,window0),(window1,window1),(window2,window2),(window3,window3)]
    overlap = [(o_const,o_const),(o_const,o_const),(o_const,o_const),(o_const,o_const)]
    
    # Define how much actual size of the frames
    scale0 = int(.527*shape[0])
    scale1 = int(.527*shape[0])
    scale2 = int(.548*shape[0])
    scale3 = int(.5625*shape[0])
    y_start_stop =[[scale0,scale0+window0/2],[scale1,scale1+window1/2],[scale2,scale2+window2/2],[scale3,scale3+window3/2]]
    
    # Search each of the windows for their classification
    hot_windows = []
    for i in range(len(Y_start_stop)):
        windows = slide_window(image, y_start_stop=y_start_stop[i], 
                            xy_window=window[i], xy_overlap=overlap[i]) 
        
        hot_windows +=  search_windows(image, windows, svc, X_scaler, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)                       

    return hot_windows

def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    if bbox_list:
        for box in bbox_list:
            # Add += 1 for all pixels inside each bbox
            # Assuming each "box" takes the form ((x1, y1), (x2, y2))
            heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1
 
    # Return updated heatmap
    return heatmap

def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img

# Define Bounding Box Class

In [4]:
# Box Class to keep track of previously detected vehicles for added stability
class BoundingBoxes:
    def __init__(self,n):
        # maintain a list of maximum size n of the last boxes seen
        self.stored_boxes = col.deque([],maxlen=n)
        # set of current boxes
        self.allboxes = []
        
    def update(self,boxes):
        self.stored_boxes.appendleft(boxes)
        newboxes = []
        for boxes in self.stored_boxes:
            newboxes += boxes
        self.allboxes = newboxes

# Define Line Class

In [5]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        self.last_fit = []
        self.y = []
        self.x = []
        self.max_keep = 5
        
        self.filled = False
        self.iterator = 0
        self.past_coefs = []
        
    # Update the stored lane values
    def update_lane(self, fit, y, x):
        self.last_fit = fit
        self.y = y
        self.x = x
        
    # Retrieve the stored lane values
    def get_lane(self):
        return self.last_fit, self.y, self.x
    
    # Cover all iterations for smoothing
    def smooth(self, new_coefs):
        # Check if enough lanes have been found
        if self.filled == False:
            # Add the new found coefficients onto the list
            self.past_coefs.append(new_coefs)
            
            # Increment the counter
            self.iterator += 1
            
            ## Smooth over however may sets of coefficients have been found so far
            out = self.get_coefs()
            
            # Check if we have sufficiently populated the list
            if self.iterator == self.max_keep:
                self.filled = True
                self.iterator = 0
                print('filled')
                
        # After the list of coefficients has been sufficiently populated
        else:
            # Change the current iterator's coefficients
            self.past_coefs[self.iterator] = new_coefs
            # Smooth of the past self.max_keep coefficients
            self.iterator += 1
            out = self.get_coefs()
            
            if self.iterator == self.max_keep:
                self.iterator = 0
            
        return out
    
    # Support function for smooth()
    def get_coefs(self):
        temp = np.copy(self.past_coefs)
        first = np.median(temp[:,0])
        second = np.median(temp[:,1])
        third = np.median(temp[:,2])
        out = np.array((first,second,third))
            
        return np.ndarray.flatten(out)

# Vehicle Detector

In [6]:
def vehicle_detector(image):
    draw_image = np.copy(image)    
    image = image.astype(np.float32)/255
    hot_windows = search(image)
    heatmap = np.zeros_like(image[:,:,0]).astype(np.float)
    boxes.update(hot_windows)
    heatmap = add_heat(heatmap, boxes.allboxes)
    heatmap  = apply_threshold(heatmap,3)
    labels = label(heatmap)
    
    return labels

# Lane Finder

In [7]:
def lane_finder(img):
    global left, right
    # undistort the image
    undist = undistort(img)
    
    # Extract the binary image
    combined = binary_extraction(undist)
     
    # Warp the image
    warped = warp(combined)
     
    # Mask it
    out = apply_mask(warped)
     
    # Generate a binary output
    binary_warped = np.array(out)
      
    # Find lane indices
    left_lane_inds, right_lane_inds = get_new_indices(binary_warped)
    
    # Where any lane pixels found?
    if (np.count_nonzero(left_lane_inds) != 0) & (np.count_nonzero(right_lane_inds) != 0):
        # Find lane fits
        left_fit, right_fit, lefty, leftx, righty, rightx = get_lane_fits(binary_warped, left_lane_inds, right_lane_inds)
        left.update_lane(left_fit, lefty, leftx)
        right.update_lane(right_fit, righty, rightx)
    else:
        # Use old fits
        left_fit, lefty, leftx = left.get_lane()
        right_fit, righty, rightx = right.get_lane()
    
    # Is this the first frame annotated
    if len(left.past_coefs) != 0:
        l_check = np.linalg.norm(np.subtract(left_fit,left.get_coefs())) < 50
        r_check = np.linalg.norm(np.subtract(right_fit,right.get_coefs())) < 50
    else:
        l_check = True
        r_check = True
    
    # If the current fit is good
    if l_check:
        # Smooth out the current fit
        smoothed_left_fit = left.smooth(left_fit)
    else:
        smoothed_left_fit = left.get_coefs()
        
    if r_check:
        # Smooth out the current fit
        smoothed_right_fit = right.smooth(right_fit)
    else:
        smoothed_right_fit = right.get_coefs()
     
    # Fit the lane pixels
    ploty, left_fitx, right_fitx = identify_lanes(binary_warped, smoothed_left_fit, smoothed_right_fit)
     
    # Find the curvature
    left_curverad, right_curverad = curvature(ploty, lefty, righty, leftx, rightx)
     
    # Draw the found lanes
    warp_zero = draw_lanes(img, ploty, left_fitx, right_fitx)
     
    # Annotate original frame with lanes
    lane = unwarp_lanes(img, warp_zero)
     
    # Annotate found lanes with text
    result = write_text(lane, left_fitx, right_fitx, left_curverad, right_curverad)
    
    return result

# Define Pipeline

In [8]:
def pipeline(img):
    found_lane = lane_finder(img)
    labels = vehicle_detector(img)
    
    output = draw_labeled_bboxes(found_lane, labels)
    
    return output

# Annotate Video

In [10]:
boxes = BoundingBoxes(n=3)
left = Line()
right = Line()

inpfile='project_video.mp4'
outfile='annotated_project.mp4'
clip = VideoFileClip(inpfile)
out_clip = clip.fl_image(lambda x:pipeline(x)) 
%time out_clip.write_videofile(outfile, audio=False)

[MoviePy] >>>> Building video annotated_project.mp4
[MoviePy] Writing video annotated_project.mp4


  0%|          | 3/1261 [00:02<15:20,  1.37it/s]

filled
filled


100%|█████████▉| 1260/1261 [14:37<00:00,  1.42it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: annotated_project.mp4 

CPU times: user 24min 7s, sys: 20.5 s, total: 24min 27s
Wall time: 14min 37s
